In [1]:
#Import the libraries

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import tensorflow
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tflearn
import json
import numpy
import random

C:\Users\Jadon\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Jadon\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
#Loading Dataset

with open('./intents.json') as file:
    data=json.load(file)
#print(data)
#print(data['intents'][0])

In [3]:
#Processing Data

words=[]
labels=[]
docs_x=[]
docs_y=[]

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds=nltk.word_tokenize(pattern) 
        words.extend(wrds)
        docs_x.append(wrds)
        #print(docs_x)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])
#print(labels)

In [4]:
#Remove duplicates from the list words

words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
#print(words)

#Sort labels
labels = sorted(labels)

In [5]:
#Neural network understands only numbers and not strings

training=[]
output=[]

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag=[]
    
    wrds = [stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])]=1
               
    training.append(bag)
    output.append(output_row)

In [6]:
#Building the model

training=numpy.array(training)
#print(training)
output=numpy.array(output)

#tensorflow.reset_default_graph()

#Classifying our data

#input layer
net = tflearn.input_data(shape=[None,len(training[0])])
print(len(training[0]))
print(len(output[0]))
#two hidden layers
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)

#output layer
net = tflearn.fully_connected(net,len(output[0]),activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1000,batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 6999  | total loss: 0.11401 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.11401 - acc: 0.9911 -- iter: 48/53
Training Step: 7000  | total loss: 1.71924 | time: 0.022s
| Adam | epoch: 1000 | loss: 1.71924 - acc: 0.8920 -- iter: 53/53
--
INFO:tensorflow:C:\Users\Jadon\Deep Learning Projects\Chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
#Predictions

def bag_of_words(s,words):
    